In [1]:
import requests
from bs4 import BeautifulSoup
import random
import csv
import re
import pandas as pd
import requests
from docx import Document
import time


user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
]

# URL của trang web cần scrap
url = 'https://www.thivien.net/B%E1%BA%A1ch-C%C6%B0-D%E1%BB%8B/author-GTDxHQP8eUITUxph4otjnA'

try:
    # Tải nội dung của trang web
    
    response = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
    response.raise_for_status()  # Ném ngoại lệ cho các mã trạng thái không tốt
    print("Tải url thành công")

    # Phân tích HTML content
    soup = BeautifulSoup(response.content, "html.parser")
    
    h1_element = soup.find("header", class_="page-header").find("h1")

    # Trích xuất và in tên "tác giả"
    if h1_element:
        author_name_text = h1_element.get_text(strip=True)
        print(author_name_text)
        
    # Trích xuất các URL từ trang web chứa từ "poem" nhưng không lấy từ "searchpoem"
    print("Trích xuất các URL")
    urls_set = set()  # Sử dụng set để loại bỏ các URL trùng lặp
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if "poem" in href and "searchpoem" not in href and "addpoem" not in href and "m-poems" not in href:
            urls_set.add(href)

    # Lưu các URL vào tệp CSV
    file_path = "D:/url_"+author_name_text+'.csv'
    with open(file_path, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["URL"])

        for url in urls_set:  # Duyệt qua tập hợp các URL không trùng lặp
            writer.writerow([url])

    print(f"Các URL đã được lưu vào {file_path}")

except requests.exceptions.RequestException as e:
    print("Lỗi khi tải URL:", e)
except Exception as e:
    print("Đã xảy ra lỗi:", e)


Tải url thành công
Bạch Cư Dị白居易
Trích xuất các URL
Các URL đã được lưu vào D:/url_Bạch Cư Dị白居易.csv


In [ ]:
# Đọc file csv chứa các URL
df = pd.read_csv(file_path)
# Khởi tạo danh sách để lưu trữ dữ liệu
data = []
# Hàm để thêm phần đầu của URL
def add_prefix(url):
    return "https://www.thivien.net" + url

# Thêm phần đầu vào từng URL
df['Full_URL'] = df['URL'].apply(add_prefix)

# Hàm để trích xuất dữ liệu từ URL và lưu vào file .docx
def extract_poem_data(url):
    user_agents_list = [
        'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
    ]
    HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
    try:
        # Delay 15 giây trước khi tải nội dung trang web
        random_sleep_time = random.randint(8, 15)
        time.sleep(random_sleep_time)
       
        
        # Tải nội dung của trang web
        print("Đang tải nội dung trang web:", url)
        #response = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()  # Ném ngoại lệ cho các mã trạng thái không tốt
   
       
        
        # Phân tích HTML content
        soup = BeautifulSoup(response.content, "html.parser")

        # Trích xuất tiêu đề
        title = soup.title.text.strip()
        print("Tiêu đề:", title)

        # Trích xuất phần chữ Hán
        han_viet_text = ""
        for p_tag in soup.find_all('p', class_='HanChinese transcriptable'):
            han_viet_text += p_tag.text.strip() + "\n"

        # Tìm phần Phiên Âm
          
        translations = []
        h4_elements = soup.find_all("h4", string=True)
        for h4_element in h4_elements:
            if h4_element.text.strip() != "Đăng nhập" and h4_element.text.strip() != "Dịch nghĩa":
                #print("Tiêu đề:", h4_element.text.strip())
                p_tag = h4_element.find_next_sibling("p")

                if p_tag:
                    #print("Nội dung p:", p_tag.text.strip()) 
                    description_content =  p_tag.text.strip()
              
         # Tìm phần Phiên Dịch
            if h4_element.text.strip() == "Dịch nghĩa":
                p_tag = h4_element.find_next_sibling("p")
                if p_tag:
                    for line in p_tag.stripped_strings:
                        content_text = line
                        translations.append(content_text)
       
            
        # Tạo dữ liệu mới
        poem_data = {
            'Tiêu đề': title,
            'Chữ Hán': han_viet_text,
            'Phiên âm': description_content,
            'Phiên dịch': '\n'.join(translations)
        }
        
        # Thêm dữ liệu vào danh sách
        data.append(poem_data)
        df_poems = pd.DataFrame(data)
        df_poems.to_csv("D:/"+author_name_text+"_poem.csv", index=False, encoding='utf-8-sig')
        
        print("ĐÃ CẬP NHẬT FILE D:/"+author_name_text+"_poem.csv")
        
        
    except requests.exceptions.RequestException as e:
        print("Lỗi khi tải URL:", e)
    except Exception as e:
        print("Đã xảy ra lỗi:", e)

# Gọi hàm để trích xuất dữ liệu từ từng URL và lưu vào file .docx
link_count = 0
random_sleep_time = 0
for url in df['Full_URL']:
    extract_poem_data(url)
    link_count += 1
    print(link_count)
    # Kiểm tra nếu biến đếm chia hết cho 5 thì nghỉ ngơi 10 giây
    if link_count % 5 == 0:           
        random_sleep_time = random.randint(15, 25)
        print(f"Đang chờ ngẫu nhiên từ {random_sleep_time} giây...")    
        time.sleep(random_sleep_time)
# Tạo DataFrame từ danh sách dữ liệu
df_poems = pd.DataFrame(data)
# Lưu DataFrame vào tệp CSV
df_poems.to_csv("D:/"+author_name_text+"_poem.csv", index=False, encoding='utf-8-sig')
print("Đã lưu dữ liệu vào file D:/"+author_name_text+"_poem.csv")

Đang tải nội dung trang web: https://www.thivien.net/B%E1%BA%A1ch-C%C6%B0-D%E1%BB%8B/D%E1%BA%A1-tuy%E1%BA%BFt/poem-uA9kRk3lnSAt0v5d5HWsAg
Tiêu đề: Bài thơ: Dạ tuyết - 夜雪 (Bạch Cư Dị - 白居易)
ĐÃ CẬP NHẬT FILE D:/Bạch Cư Dị白居易_poem.csv
1
Đang tải nội dung trang web: https://www.thivien.net/B%E1%BA%A1ch-C%C6%B0-D%E1%BB%8B/C%E1%BA%A3m-k%C3%ADnh/poem-YR1JO8YsFFO7R_h7lgqsNQ
Tiêu đề: Bài thơ: Cảm kính - 感鏡 (Bạch Cư Dị - 白居易)
ĐÃ CẬP NHẬT FILE D:/Bạch Cư Dị白居易_poem.csv
2
Đang tải nội dung trang web: https://www.thivien.net/B%E1%BA%A1ch-C%C6%B0-D%E1%BB%8B/H%E1%BB%93ng-tuy%E1%BA%BFn-th%E1%BA%A3m/poem-10mrMkkxVSBUkfRAAvCy-g
Tiêu đề: Bài thơ: Hồng tuyến thảm - 紅線毯 (Bạch Cư Dị - 白居易)
ĐÃ CẬP NHẬT FILE D:/Bạch Cư Dị白居易_poem.csv
3
Đang tải nội dung trang web: https://www.thivien.net/B%E1%BA%A1ch-C%C6%B0-D%E1%BB%8B/Ngo%E1%BA%A1n-t%C3%A2n-%C4%91%C3%ACnh-th%E1%BB%A5-nh%C3%A2n-v%E1%BB%8Bnh-s%E1%BB%9F-ho%C3%A0i/poem-teLy8EEaljbW3gVSsXwTXA
Tiêu đề: Bài thơ: Ngoạn tân đình thụ, nhân vịnh sở hoài - 玩新庭樹，因詠所懷 (Bạ

In [ ]:
# Đọc dữ liệu từ file CSV
df = pd.read_csv("D:/"+author_name_text+"_poem.csv")

# Tạo một cột mới để lưu phần ghép
df['Lồng ghép1'] = ""
df['Nghĩa tựa'] = ""

df['Chữ Hán'] = df['Chữ Hán'].str.replace("，", ",")
df['Chữ Hán'] = df['Chữ Hán'].str.replace("。", ",")
df['Chữ Hán'] = df['Chữ Hán'].str.replace("？", ",")
df['Chữ Hán'] = df['Chữ Hán'].str.replace("！", ",")

df['Phiên âm'] = df['Phiên âm'].str.replace(",", " ")
df['Phiên âm'] = df['Phiên âm'].str.replace("?", " ")
df['Phiên âm'] = df['Phiên âm'].str.replace(".", " ")
df['Phiên âm'] = df['Phiên âm'].str.replace("!", " ")
df['Phiên âm'] = df['Phiên âm'].str.replace(":", " ")
df['Phiên âm'] = df['Phiên âm'].str.replace(")", " ")
df['Phiên âm'] = df['Phiên âm'].str.replace(" (", "-")
df['Phiên âm'] = df['Phiên âm'].str.replace("  ", " ")

df['Tiêu đề'] = df['Tiêu đề'].str.replace("Bài thơ: ", "")
df['Tiêu đề'] = df['Tiêu đề'].str.replace("-", "")
df['Tiêu đề'] = df['Tiêu đề'].str.replace("一", "")
df['Tiêu đề'] = df['Tiêu đề'].str.replace(")", "")
df['Tiêu đề'] = df['Tiêu đề'].str.replace("(", "")
# Duyệt qua mỗi dòng của DataFrame
for index, row in df.iterrows():
    chinese_parts = row['Chữ Hán'].split(",")
    phonetic_parts = row['Phiên âm']
    
    if isinstance(row['Phiên dịch'], str):
        translate_part= row['Phiên dịch'].split('\n')
    else:
        translate_part= row['Chữ Hán'].split(",")
    i=0
    
    # Tiếp tục xử lý như bình thường nếu số lượng phần tử trong mỗi phần tách của chinese_parts đều bằng nhau
    merged_text = ""
    n=0
    
    for i in range(len(chinese_parts)):
        han=chinese_parts[i].strip()
        k=len(chinese_parts[i].strip())
        
        
        phienam=phonetic_parts.split()[n:n+k]
       
        #print(k,n*k,(n+1)*k,phienam)
        phienam_str = ' '.join(phienam)
        
        if i+1 < len(translate_part):
            dich = translate_part[i+1].strip()
            print(han,'\n',phienam_str,'\n',dich)
            merged_text += f"{han}\n{phienam_str}\n{dich}\n\n"
        else:
            dich = ""
            merged_text += f"{han}\n{phienam_str}\n\n"
        n=n+k
    # Gán chuỗi lồng ghép mới vào cột 'Lồng ghép1' tương ứng
    df.at[index, 'Lồng ghép1'] = merged_text
     # Gán tựa đề
    df.at[index, 'Nghĩa tựa'] = translate_part[0].strip()
# Lưu DataFrame đã được cập nhật vào file CSV
df.to_csv("D:/updated_"+author_name_text+"_poem.csv", index=False,encoding='utf-8-sig')
print("Đã lưu dữ liệu đã được cập nhật vào file D:/updated_"+author_name_text+"_poem.csv")


In [ ]:
#convert csv to docx
import pandas as pd
from docx import Document

# Đọc dữ liệu từ file CSV
df = pd.read_csv("D:/updated_"+author_name_text+"_poem.csv")

# Tạo một đối tượng Document mới
doc = Document()

# Duyệt qua mỗi dòng của DataFrame
for index, row in df.iterrows():
    # Tiêu đề và nghĩa tựa
    title = row['Tiêu đề']
    meaning = row['Nghĩa tựa']
    
    # Tạo Heading 1 cho tiêu đề và nghĩa tựa
    doc.add_heading(f"{title} - {meaning}", level=1)
    
    # Lồng ghép1
    lồng_ghép = row['Lồng ghép1']
    
    # Thêm lồng ghép vào tài liệu
    doc.add_paragraph(lồng_ghép)

# Lưu tài liệu vào file DOCX
doc.save("D:/"+author_name_text+"_poem.docx")
print("Đã lưu tài liệu vào file D:/"+author_name_text+"_poem.docx")